RESULT of this experiment
USE AREA intepolation for shrinking and use LANCZOs4/CUBIC interpolation for expanding

I want to check the conversion of small size vs the averages of 4x4. Which has better psnr to the orignal image. Also I want to the the increase of image size by 4 times. Which gives better result.

In [ ]:
from matplotlib import pyplot as plt
from PIL import Image
import numpy as np
import pandas as pd
import time
import cv2
from os import walk

In [ ]:
openmode=cv2.IMREAD_COLOR
dir_path='./samples/'
output_path='./test_size_results/'

In [ ]:
files = []
for (dir_path, dir_names, file_names) in walk(dir_path):
    files.extend([dir_path+"/"+f for f in file_names])
files

In [ ]:
len(files)

making dataframe

In [ ]:
CV2INTERS=[cv2.INTER_AREA,
# cv2.INTER_BITS,
# cv2.INTER_BITS2,
# cv2.INTER_CUBIC,
# cv2.INTER_LANCZOS4,
# cv2.INTER_LINEAR,
# cv2.INTER_LINEAR_EXACT,
# cv2.INTER_NEAREST
]
CV2INTERL=[
  # cv2.INTER_AREA,
# cv2.INTER_BITS,
# cv2.INTER_BITS2,
cv2.INTER_CUBIC,
cv2.INTER_LANCZOS4,
# cv2.INTER_LINEAR,
# cv2.INTER_LINEAR_EXACT,
# cv2.INTER_NEAREST
]
CV2INTER_STRS=['AREA',
              # 'BITS', 'BITS2', 
              # 'CUBIC', 'LANCZOS4',
              # 'LINEAR', 'LINEAR_EXACT', 'NEAREST'
              ]
CV2INTER_STRL=[
  # 'AREA',
              # 'BITS', 'BITS2', 
              'CUBIC', 'LANCZOS4',
              # 'LINEAR', 'LINEAR_EXACT', 'NEAREST'
              ]

In [ ]:
columns=['file','size','small_method','small_t','small_size','large_method','large_t', 'psnr']

In [ ]:
df=pd.DataFrame(columns=columns)

In [ ]:
zoomx=8
shrink=1/zoomx
enlarge=zoomx

testing on one image

In [ ]:
imarr=cv2.imread('./samples//large/miku.jpg', openmode)
Image.fromarray(imarr[:,:,::-1])

In [ ]:
imarrsmall=cv2.resize(imarr, None, fx=shrink, fy=shrink, interpolation=cv2.INTER_AREA)


In [ ]:
Image.fromarray(imarrsmall[:,:,::-1])

In [ ]:
startx = 100
starty = 100
width = 100
height = 100

for i in range(width):
    for j in range(height):
        imarrsmall[startx+i][starty+j] = 100

In [ ]:
Image.fromarray(imarrsmall[:,:,::-1])

In [ ]:
imarrlarge=cv2.resize(imarrsmall, None, fx=enlarge, fy=enlarge, interpolation=cv2.INTER_CUBIC)


In [ ]:
Image.fromarray(imarrlarge[:,:,::-1])

In [ ]:
for f in files:
  imarr=cv2.imread(f, openmode)
  if imarr is None or imarr.shape[0]%4!=0 or  imarr.shape[1]%4!=0 :
    print("Skipping:",f)
    continue
  for idxs, sm in enumerate(CV2INTERS):
    small_t=time.time()
    imarrsmall=cv2.resize(imarr, None, fx=shrink, fy=shrink, interpolation=sm)
    small_t=time.time()-small_t
    for idxl, lm in enumerate(CV2INTERL):
      large_t=time.time()
      imarrlarge=cv2.resize(imarrsmall, None, fx=enlarge, fy=enlarge, interpolation=lm)
      large_t=time.time()-large_t
      # add to the data frame
      df.loc[len(df)]=[f,imarr.shape,CV2INTER_STRS[idxs],small_t ,imarrsmall.shape,CV2INTER_STRL[idxl],large_t,cv2.PSNR(imarr,imarrlarge)]
    # save after every small method
    df.to_csv(output_path+'test_size_rgb_specific.csv', index=False)
      

ab har kisi ka maximum and 2nd maximum nika lain gy

In [ ]:
dfmax=pd.DataFrame(columns=columns)

In [ ]:
df

In [ ]:
# take out the maximum and 2nd maximum psnr methods for each files
for f in files:
  print(f)
  temp=df[df['file']==f]
  if len(temp)==0:
    continue
  klargest=temp['psnr'].nlargest(2)
  dfmax=pd.concat([dfmax,temp[temp['psnr']==klargest.iloc[0]]])
  # pd.concat([dfmax,temp[temp['psnr']==klargest.iloc[1]]])
  

In [ ]:
dfmax

In [ ]:
len(dfmax)

In [ ]:
dfmax.groupby(['small_method','large_method']).count()


In [ ]:
dfmax.to_csv(output_path+'test_size_rgb_specific_max.csv', index=False)

In [ ]:
dfmax=pd.DataFrame(columns=columns)

In [ ]:
# take out the maximum and 2nd maximum psnr methods for each files
for f in files:
  # print(f)
  temp=df[df['file']==f]
  if len(temp)==0:
    continue
  klargest=temp['psnr'].nlargest(2)
  dfmax=pd.concat([dfmax,temp[temp['psnr']==klargest.iloc[1]]])
  # pd.concat([dfmax,temp[temp['psnr']==klargest.iloc[1]]])
  

In [ ]:
len(dfmax)

In [ ]:
dfmax.groupby(['small_method','large_method']).count()


In [ ]:
dfmax.to_csv(output_path+'test_size_rgb_specific_2ndmax.csv', index=False)

In [ ]:
dfmax=pd.DataFrame(columns=columns)

In [ ]:
# take out the maximum and 2nd maximum psnr methods for each files
for f in files:
  print(f)
  temp=df[df['file']==f]
  if len(temp)==0:
    continue
  klargest=temp['psnr'].nlargest(2)
  dfmax=pd.concat([dfmax,temp[temp['psnr']==klargest.iloc[0]]])
  dfmax=pd.concat([dfmax,temp[temp['psnr']==klargest.iloc[1]]])
  

In [ ]:
len(dfmax)

In [ ]:
dfmax.groupby(['small_method','large_method']).count()

In [ ]:
# dfmax.groupby(['small_method','large_method']).count().to_csv(output_path+'test_size_rgb_count.csv', index=False)